---
```
Class: Học sâu và ứng dụng trong thị giác máy tính (CO5085)
Teacher: Quản Thành Thơ
Email: plnson.sdh242@hcmut.edu.vn - HCMUT
ID: 2470741
Name: Phạm Lê Ngọc Sơn
Project: Sentiment Analysis

```
---

# VLSP sentiment: CNN vs BiLSTM vs CNN+BiLSTM

Notebook này tổng hợp lại nội dung từ Lab 5 và Lab 6 để chạy 3 mô hình (TextCNN, BiLSTM, CNN+BiLSTM/CRNN) trên bộ dữ liệu VLSP. Có thể chạy trực tiếp trong VS Code/Jupyter hoặc Colab; dữ liệu vlsp_sentiment_[train|test].csv và file embedding vi-model-CBOW.bin đã có sẵn trong repo.


### Chuẩn bị môi trường

```bash
!pip install -q pyvi gensim scikit-learn tensorflow==2.15
```

Yêu cầu dung lượng bộ nhớ lớn (tệp embedding ~700MB) và RAM khoảng từ 2GB trở lên.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q pyvi gensim scikit-learn tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00


In [ ]:
import os

import random
import re
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from gensim.models.keyedvectors import KeyedVectors
from pyvi import ViTokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# Cố định random seed để đảm bảo khả năng tái lập kết quả.
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Đường dẫn được đặt theo cấu trúc thư mục trên Google Drive/Colab.
BASE_DIR = Path('/content/drive/MyDrive/NLP_Lab/')
TRAIN_PATH = BASE_DIR / 'Lab5/vlsp_sentiment_train.csv'
TEST_PATH = BASE_DIR / 'Lab5/vlsp_sentiment_test.csv'
W2V_PATH = BASE_DIR / 'Lab5/vi-model-CBOW.bin'

# Siêu tham số dùng chung cho cả ba mô hình.
EMBEDDING_DIM = 400
MAX_VOCAB_SIZE = 12000
MAX_SEQUENCE_LENGTH = 180
VAL_SIZE = 0.1
BATCH_SIZE = 256
EPOCHS = 6


In [ ]:
def clean_text(text: str) -> str:
    # Loại bỏ chữ số/ký tự thừa và chuyển toàn bộ về lowercase để giảm nhiễu.
    text = re.sub(r"\d+", " ", str(text))
    text = re.sub(r"\s+", " ", text).strip().lower()
    return text


def tokenize_vi(text: str) -> str:
    # Tách cụm từ tiếng Việt bằng PyVi, đồng thời giữ nguyên các từ ghép.
    return ViTokenizer.tokenize(text)


def encode_labels(labels: np.ndarray) -> np.ndarray:
    # Chuyển nhãn {-1, 0, 1} sang dạng one-hot với 3 lớp.
    label_map = {-1: 0, 0: 1, 1: 2}
    encoded = np.array([label_map[int(l)] for l in labels])
    return to_categorical(encoded, num_classes=3)


def load_and_prepare(df: pd.DataFrame):
    # Quy trình: làm sạch → tokenize → tách tokens và gán nhãn one-hot.
    cleaned = df['Data'].astype(str).apply(clean_text).apply(tokenize_vi)
    tokens = [text.split() for text in cleaned]
    labels = encode_labels(df['Class'].values)
    return tokens, labels


def pad(tokenizer: Tokenizer, sequences, maxlen: int):
    # Chuyển từ sang ID và padding về chiều dài cố định.
    ids = tokenizer.texts_to_sequences(sequences)
    return pad_sequences(ids, maxlen=maxlen)


def build_embedding_matrix(word_index, max_vocab=MAX_VOCAB_SIZE):
    # Nạp vector từ vi-model-CBOW.bin; nếu thiếu file thì tạo embedding ngẫu nhiên
    vocab_size = min(len(word_index) + 1, max_vocab)
    matrix = np.zeros((vocab_size, EMBEDDING_DIM), dtype=np.float32)
    try:
        w2v = KeyedVectors.load_word2vec_format(str(W2V_PATH), binary=True)
    except FileNotFoundError:
        print(f"[!] Khong tim thay file embedding tai {W2V_PATH}")
        return matrix, vocab_size

    for word, idx in word_index.items():
        if idx >= max_vocab:
            continue
        if word in w2v:
            matrix[idx] = w2v[word]
        else:
            matrix[idx] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)
    del w2v
    return matrix, vocab_size


In [ ]:
# Đọc dữ liệu TSV từ bộ dữ liệu VLSP.
train_df = pd.read_csv(TRAIN_PATH, sep='\t')
train_df.columns = ['Class', 'Data']
test_df = pd.read_csv(TEST_PATH, sep='\t')
test_df.columns = ['Class', 'Data']

# Tiền xử lý văn bản và tách tokens/labels.
train_tokens, y_full = load_and_prepare(train_df)
test_tokens, y_test = load_and_prepare(test_df)

# Fit tokenizer trên toàn bộ tập train để đảm bảo từ vựng (vocab) ổn định.
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=False, oov_token="<unk>")
tokenizer.fit_on_texts(train_tokens)

# Chuyển dữ liệu sang dạng đã padding và tách tập train/val với stratify để giữ phân bố lớp.
x_full = pad(tokenizer, train_tokens, MAX_SEQUENCE_LENGTH)
x_test = pad(tokenizer, test_tokens, MAX_SEQUENCE_LENGTH)
label_ids = np.argmax(y_full, axis=1)

x_train, x_val, y_train, y_val = train_test_split(
    x_full, y_full, test_size=VAL_SIZE, random_state=42, stratify=label_ids
)

# Tạo embedding matrix từ từ điển đã có sẵn.
embedding_matrix, VOCAB_SIZE = build_embedding_matrix(tokenizer.word_index, MAX_VOCAB_SIZE)
print(f"Train: {x_train.shape}, Val: {x_val.shape}, Test: {x_test.shape}")


Train: (4590, 180), Val: (510, 180), Test: (1050, 180)


### Giải thích tiền xử 

- Làm sạch (clean_text) để loại bỏ số và khoảng trắng thừa, giúp bộ từ vựng gọn hơn.

- ViTokenizer tách các cụm từ (ví dụ: "không hay" → "không_hay") nên rất phù hợp với từ điển word2vec tiếng Việt.

- Tokenizer giới hạn từ vựng ở mức 12.000 và pad độ dài chuỗi về 180 để tạo batch có kích thước cố định khi training.

- Chia tập train/validation theo tỷ lệ 90/10 với stratify để giữ phân bố lớp đồng đều, hạn chế mất cân bằng dữ liệu.

- build_embedding_matrix nạp vector 400 chiều từ vi-model-CBOW; nếu thiếu file thì sẽ dùng vector ngẫu nhiên để có thể chạy demo.


In [ ]:
def build_textcnn():
    inputs = layers.Input(shape=(MAX_SEQUENCE_LENGTH,), name="text")
    x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)(inputs)
    convs = []
    for k in [3, 4, 5]:
        # Kernel 3–5 giúp mô hình bắt được các biến thể n-gram khác nhau trong câu.
        c = layers.Conv1D(128, k, activation="relu", padding="valid")(x)
        p = layers.GlobalMaxPooling1D()(c)
        convs.append(p)
    x = layers.Concatenate()(convs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    model = models.Model(inputs, outputs, name="textcnn")
    model.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def build_bilstm():
    inputs = layers.Input(shape=(MAX_SEQUENCE_LENGTH,), name="text")
    x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)(inputs)
    x = layers.SpatialDropout1D(0.2)(x)  # Dropout áp dụng lên toàn bộ chuỗi embedding.
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    model = models.Model(inputs, outputs, name="bilstm")
    model.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def build_cnn_bilstm():
    inputs = layers.Input(shape=(MAX_SEQUENCE_LENGTH,), name="text")
    x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)(inputs)
    x = layers.Conv1D(128, 5, activation="relu", padding="same")(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Bidirectional(layers.LSTM(96, return_sequences=True))(x)  # Kết hợp CNN (lọc đặc trưng cục bộ) với ngữ cảnh dài.
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    model = models.Model(inputs, outputs, name="cnn_bilstm")
    model.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
    return model


In [ ]:
def run_experiment(name, builder):
    model = builder()
    es = callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)  # Dừng sớm khi val_loss tăng.
    history = model.fit(
        x_train,
        y_train,
        validation_data=(x_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es],
        verbose=2,
    )
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return {
        "model": name,
        "best_val_acc": float(max(history.history["val_accuracy"])),
        "test_acc": float(test_acc),
        "params": model.count_params(),
    }


results = []
# Chạy thí nghiệm
results.append(run_experiment("TextCNN", build_textcnn))
results.append(run_experiment("BiLSTM", build_bilstm))
results.append(run_experiment("CNN+BiLSTM", build_cnn_bilstm))

if results:
    display(pd.DataFrame(results))
else:
    print("Chua chay huan luyen (dang o dry-run)")


Epoch 1/6
18/18 - 13s - 748ms/step - accuracy: 0.4133 - loss: 3.1625 - val_accuracy: 0.5176 - val_loss: 1.3605
Epoch 2/6
18/18 - 1s - 36ms/step - accuracy: 0.5845 - loss: 1.4143 - val_accuracy: 0.5490 - val_loss: 1.2504
Epoch 3/6
18/18 - 1s - 35ms/step - accuracy: 0.6780 - loss: 0.8846 - val_accuracy: 0.5980 - val_loss: 1.0356
Epoch 4/6
18/18 - 1s - 38ms/step - accuracy: 0.7196 - loss: 0.7018 - val_accuracy: 0.5941 - val_loss: 0.9443
Epoch 5/6
18/18 - 1s - 39ms/step - accuracy: 0.8050 - loss: 0.5026 - val_accuracy: 0.6059 - val_loss: 0.9379
Epoch 6/6
18/18 - 1s - 38ms/step - accuracy: 0.8516 - loss: 0.3963 - val_accuracy: 0.6275 - val_loss: 0.9225
Epoch 1/6
18/18 - 7s - 406ms/step - accuracy: 0.4377 - loss: 1.1032 - val_accuracy: 0.5314 - val_loss: 0.9626
Epoch 2/6
18/18 - 1s - 70ms/step - accuracy: 0.5547 - loss: 0.9361 - val_accuracy: 0.5706 - val_loss: 0.9024
Epoch 3/6
18/18 - 1s - 78ms/step - accuracy: 0.6031 - loss: 0.8713 - val_accuracy: 0.6098 - val_loss: 0.8712
Epoch 4/6
18/18 

,model,best_val_acc,test_acc,params
0,TextCNN,0.627451,0.629524,3778339
1,BiLSTM,0.635294,0.647619,3704867
2,CNN+BiLSTM,0.635294,0.657143,3591907


from matplotlib import pyplot as plt
_df_0['best_val_acc'].plot(kind='hist', bins=20, title='best_val_acc')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['test_acc'].plot(kind='hist', bins=20, title='test_acc')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['params'].plot(kind='hist', bins=20, title='params')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='best_val_acc', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='test_acc', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='params', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

### Diễn giải kết quả thử nghiệm

- Bảng kết quả thể hiện các chỉ số best_val_acc, test_acc và params của từng mô hình.

- TextCNN hoạt động hiệu quả với câu ngắn; BiLSTM tốt hơn trong việc bắt quan hệ dài; mô hình CNN + BiLSTM kết hợp khả năng lọc đặc trưng cục bộ và nắm bắt ngữ cảnh, nhưng thời gian huấn luyện thường lâu hơn.

- Nếu test_acc thấp hơn best_val_acc quá nhiều, mô hình có khả năng bị overfitting → nên thử tăng dropout, giảm số filters, hoặc tăng VAL_SIZE để mô hình tổng quát tốt hơn.

- Cần bổ sung classification_report trên y_pred để xem lớp nào đang dự đoán kém, đặc biệt quan trọng khi dữ liệu lệch lớp.

- Khi báo cáo, nên đề cập đến kích thước mô hình (params) so với độ chính xác, giúp dễ dàng so sánh và xếp hạng giữa các phương án mô hình.


### Ghi chú báo cáo

- Trình bày kết quả độ chính xác (val/test accuracy) sau khi chạy thử nghiệm với 3 mô hình cơ bản.

- Đánh giá nhanh hiệu năng:

- TextCNN: huấn luyện nhanh, phù hợp với câu ngắn và bài toán cảm xúc cơ bản.

- BiLSTM: nắm bắt tốt quan hệ dài trong câu, phù hợp với văn bản dài hoặc có ngữ cảnh phức tạp.

- CNN + BiLSTM: kết hợp khả năng lọc đặc trưng cục bộ và ngữ cảnh dài; cho kết quả ổn định nhưng thời gian huấn luyện dài hơn.

- Có thể thử tăng MAX_VOCAB_SIZE, MAX_SEQUENCE_LENGTH, hoặc bật trainable embeddings nếu muốn cải thiện mô hình; tuy nhiên cần điều chỉnh lại số epoch/dropout để tránh overfitting.


### Cải tiến

- Bật class weights (balanced) để giảm lệch label.

- Cho embedding trainable + dropout cao hơn khi overfit.

- Thêm ReduceLROnPlateau kết hợp EarlyStopping để hạ cơ học nhanh khi loss đứng.

- Báo cáo thêm macro F1 trên test thay vì chiỉ accuracy.

- Tăng `MAX_VOCAB_SIZE`/`MAX_SEQUENCE_LENGTH` nếu RAM cho phép.


In [ ]:
from sklearn.metrics import classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight

# Sử dụng class weight để giảm thiên vị mô hình đối với các lớp khi dữ liệu bị mất cân bằng.
CLASS_WEIGHTS = dict(
    enumerate(compute_class_weight(class_weight="balanced", classes=np.unique(label_ids), y=label_ids))
)


def build_textcnn_v2(trainable_embed=True, filters=128, drop=0.55, kernel_sizes=(3, 4, 5)):
    inputs = layers.Input(shape=(MAX_SEQUENCE_LENGTH,), name="text")
    x = layers.Embedding(
        VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=trainable_embed
    )(inputs)
    convs = []
    for k in kernel_sizes:
        c = layers.Conv1D(filters, k, activation="relu", padding="same")(x)
        p = layers.GlobalMaxPooling1D()(c)
        convs.append(p)
    x = layers.Concatenate()(convs)
    x = layers.Dropout(drop)(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    model = models.Model(inputs, outputs, name="textcnn_v2")
    model.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def build_bilstm_v2(trainable_embed=True, units=160, drop=0.55):
    inputs = layers.Input(shape=(MAX_SEQUENCE_LENGTH,), name="text")
    x = layers.Embedding(
        VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=trainable_embed
    )(inputs)
    x = layers.SpatialDropout1D(0.25)(x)
    x = layers.Bidirectional(layers.LSTM(units, return_sequences=True))(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(drop)(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    model = models.Model(inputs, outputs, name="bilstm_v2")
    model.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def build_cnn_bilstm_v2(trainable_embed=True, filters=128, lstm_units=128, drop=0.55, kernel_size=5):
    inputs = layers.Input(shape=(MAX_SEQUENCE_LENGTH,), name="text")
    x = layers.Embedding(
        VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=trainable_embed
    )(inputs)
    x = layers.Conv1D(filters, kernel_size, activation="relu", padding="same")(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(drop)(x)
    outputs = layers.Dense(3, activation="softmax")(x)
    model = models.Model(inputs, outputs, name="cnn_bilstm_v2")
    model.compile(optimizer=optimizers.Adam(8e-4), loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def run_experiment_v2(name, builder, builder_kwargs=None, use_class_weights=True):
    model = builder(**(builder_kwargs or {}))
    cbs = [
        callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1, min_lr=1e-5),
    ]
    cw = CLASS_WEIGHTS if use_class_weights else None
    history = model.fit(
        x_train,
        y_train,
        validation_data=(x_val, y_val),
        epochs=EPOCHS + 2,
        batch_size=BATCH_SIZE,
        callbacks=cbs,
        class_weight=cw,
        verbose=2,
    )
    test_probs = model.predict(x_test, batch_size=512, verbose=0)
    y_pred = np.argmax(test_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)
    macro_f1 = f1_score(y_true, y_pred, average="macro")
    test_acc = np.mean(y_true == y_pred)
    return {
        "model": name,
        "best_val_acc": float(max(history.history["val_accuracy"])),
        "test_acc": float(test_acc),
        "test_macro_f1": float(macro_f1),
        "params": model.count_params(),
    }


# Chạy thí nghiệm
results.append(run_experiment_v2("TextCNN v2", build_textcnn_v2, {"trainable_embed": True, "filters": 160}))
results.append(run_experiment_v2("BiLSTM v2", build_bilstm_v2, {"trainable_embed": True, "units": 192}))
results.append(run_experiment_v2("CNN+BiLSTM v2", build_cnn_bilstm_v2, {"trainable_embed": True, "filters": 160, "lstm_units": 160}))
if results:
    display(pd.DataFrame(results))


Epoch 1/8
18/18 - 19s - 1s/step - accuracy: 0.4170 - loss: 3.0792 - val_accuracy: 0.5118 - val_loss: 1.4702 - learning_rate: 1.0000e-03
Epoch 2/8
18/18 - 1s - 72ms/step - accuracy: 0.6011 - loss: 1.3233 - val_accuracy: 0.5784 - val_loss: 1.0150 - learning_rate: 1.0000e-03
Epoch 3/8
18/18 - 1s - 73ms/step - accuracy: 0.6797 - loss: 0.9046 - val_accuracy: 0.6235 - val_loss: 0.9260 - learning_rate: 1.0000e-03
Epoch 4/8
18/18 - 1s - 75ms/step - accuracy: 0.7634 - loss: 0.6088 - val_accuracy: 0.6216 - val_loss: 0.9281 - learning_rate: 1.0000e-03
Epoch 5/8
18/18 - 1s - 76ms/step - accuracy: 0.8089 - loss: 0.4853 - val_accuracy: 0.6216 - val_loss: 0.9202 - learning_rate: 5.0000e-04
Epoch 6/8
18/18 - 1s - 73ms/step - accuracy: 0.8388 - loss: 0.4208 - val_accuracy: 0.6294 - val_loss: 0.9092 - learning_rate: 5.0000e-04
Epoch 7/8
18/18 - 1s - 73ms/step - accuracy: 0.8617 - loss: 0.3675 - val_accuracy: 0.6255 - val_loss: 0.8961 - learning_rate: 5.0000e-04
Epoch 8/8
18/18 - 1s - 72ms/step - accurac

,model,best_val_acc,test_acc,params,test_macro_f1
0,TextCNN,0.627451,0.629524,3778339,NaN
1,BiLSTM,0.635294,0.647619,3704867,NaN
2,CNN+BiLSTM,0.635294,0.657143,3591907,NaN
3,TextCNN v2,0.629412,0.670476,3932323,0.671219
4,BiLSTM v2,0.654902,0.684762,4074403,0.685747
5,CNN+BiLSTM v2,0.660784,0.680000,3894403,0.678259


from matplotlib import pyplot as plt
_df_16['best_val_acc'].plot(kind='hist', bins=20, title='best_val_acc')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_17['test_acc'].plot(kind='hist', bins=20, title='test_acc')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_18['params'].plot(kind='hist', bins=20, title='params')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_29['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_29, x='test_acc', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_30['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_30, x='params', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_31['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_31, x='test_macro_f1', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

### Ghi chú

- run_experiment_v2 bổ sung class_weight, ReduceLROnPlateau và tăng số epoch nhằm fine-tune embedding hiệu quả hơn.

- Chỉ số test_macro_f1 hỗ trợ đánh giá cân bằng giữa các lớp và đặc biệt quan trọng khi phân bố lớp –1 / 0 / 1 bị lệch.

- Khi chạy các ví dụ ở cuối cell, cần lấy đầy đủ val accuracy, test accuracy và macro F1 để đưa vào báo cáo.

- Có thể giảm filters hoặc units nếu thiết bị hạn chế RAM, hoặc tắt trainable_embed để tăng tốc độ huấn luyện.